## DCN 论文理解
### 1.介绍
&emsp;&emsp;CTR预估全称是 Click Through Rate，即展示给用户的广告或者商品，估计用户点击的概率。传统的CTR预估模型需要大量的特征工程，耗时耗力；引入DNN后，依靠神经网络强大的学习能力，可以一定程度上实现自动学习的特征组合，但是DNN的缺点在于隐式的学习特征组合带来的不可解释性，以及低效率的学习(并不是所有的特征组合都是有用的)。

&emsp;&emsp;DCN 全称deep & cross network，DCN在学习特定阶数组合特征的时候效率非常高，而且同样不需要特征工程，引入额外的复杂度也很小。

### 2.相关工作
&emsp;&emsp;最开始FM使用隐向量的内积来建模组合特征；FFM在此基础上引入field的概念，针对不同的field上使用不同隐向量，但是这两者都是针对低阶的特征组合进行建模的。DNN可以对高纬组合特征进行建模，含义非常难以理解，设计一种DNN的特定网络结构来改善DNN，使其学习起来更加高效。

### 3.DCN特点
1. 使用cross network，在每一层定应用feature crossing。高效的学习了特定阶(bounded degree)组合特征，不需要人工特征工程。
2. 网络结构简单而且高效。多项式复杂度由layer depth决定。
3. 相比于DNN，DCN的logloss更低，而且参数的数量将近少了一个数量级。

### 4.DCN
![DCN图](./pic/DCN架构.jpg)

&emsp;&emsp;DCN架构如图所示：最开始是embedding and stacking layer，然后是并行的cross network 和deep network，最后是combination layer将cross network和deep network的结果组合得到output。

#### 4.1 embedding and stacking layer
&emsp;&emsp;在ctr预估中，输入的大部分特征都是类别型特征，one-hot处理之后的输入特征维度非常高，所以通过embedding大幅度降低输入的维度，将这些binary features转化成dense vectors with real values。

&emsp;&emsp;embedding 操作其实就是用一个矩阵和one-hot之后的输入相乘，也可以看成是一次查询(lookup)，这个embedding矩阵跟网络中的其他参数一样，是需要随着网络一起学习的。

&emsp;&emsp;处理完了类别型特征，还有连续型特征没有处理，所以将连续型特规范化之后，和embedding vector stacking到一起，就得到了原始的输入。

![DCN图](./pic/embedding_stacking.jpg)

$$x_{embed,i} = W_{embed,i}x_{i} \tag{1}$$

&emsp;&emsp;$x_{embed,i}$是embedding vector，$x_{i}$是第i个类别的0/1输入，$w_{embed,i}$是第i个特征的embedding矩阵。

$$x_{0} = [x_{embed,1}^{T},...,x_{embed,k}^{T},x_{dense}^{T}] \tag{2}$$

&emsp;&emsp;$x_{0}$是最终DCN的输入。

#### 4.2 cross network
&emsp;&emsp;cross network 是论文的核心，被设计用来高效的学习组合特征。

$$x_{l+1} = x_{0}x_{l}^{T}w_{l} + b_{l} + x_{l} = f(x_{l},w_{l},b_{l}) + x_{l} \tag{3}$$

&emsp;&emsp;其中$x_{l},x_{l+1}$分别是第l层，第l+1层cross layer的输出，$w_{l},b_{l}$是这两层之间的连接参数，均为列向量。

![cross_layer图](./pic/cross_layer.jpg)

**交叉特征高度交互**

&emsp;&emsp;交叉网络的特殊结构使交叉特征的程度随着层深度的增加而增大，对于$x_{0}$和l层交叉网络，最高的阶数是l+1，事实上，交叉网络包含了从1到l+1的所有交叉项。

**复杂度分析**

&emsp;&emsp;$L_{c}$表示交叉层数，d表示输入维度，然后，参数的数量为: $d * l_{c} * 2$

&emsp;&emsp;交叉网络的时间和空间复杂度在输入维度上是线性的，这种效率得益于$x_{0}x_{l}^{T}$是一个标量，它使得能够在不计算或存储这个那个矩阵的情况下生成所有交叉项。

&emsp;&emsp;交叉网络的少量参数限制了模型能力，为了捕捉高度非线性的相互作用，并行的引入一个深度网络。

#### 4.3 deep network
&emsp;&emsp;深度网络就是一个全连接的前馈神经网络，每个深度层具有如下公式:
$$h_{l+1} = f(w_{l}h_{l} + b_{l}) \tag{4}$$

#### 4.4 combination layer
&emsp;&emsp;融合层连接连个网络并将连接向量输入到标准的逻辑回归函数。

$$p = sigmoid([x_{L_{1}}^{T},h_{L_{2}}^{T}]w_{logits}) \tag{5}$$

&emsp;&emsp;loss function:
$$loss = -\frac{1}{N}\sum_{i=1}^{N}{y_{i}log(p_{i}) + (1-y_{i})log(1-p_{i}) + \lambda \sum_{l}||w_{l}||^{2}} \tag{6}$$

### 5. 交叉网络结构分析
#### 5.1 高阶分析
&emsp;&emsp;论文中的证明过程很难理解，举一个具体的例子，假设cross 有一层，$X_{0} = [x_{0,1},x_{0,2}]^{T}$，简便处理$b_{i}$ =0。

$$x_{1} = x_{0}x_{0}^{T}w_{0} + x_{0} = 
\left[
\begin{matrix}
x_{0,1} \\
x_{0,2} \\
\end{matrix}
\right]
\left[
\begin{matrix}
x_{0,1} & x_{0,2} \\
\end{matrix}
\right]
\left[
\begin{matrix}
w_{0,1} \\
w_{0,2} \\
\end{matrix}
\right]
+ 
\left[
\begin{matrix}
x_{0,1} \\
x_{0,2} \\
\end{matrix}
\right]
=
\left[
\begin{matrix}
w_{0,1}x_{0,1}^{2} + w_{0,2}x_{0,1}x_{0,2} + x_{0,1} \\
w_{0,1}x_{0,2}x_{0,1} + w_{0,2}x_{0,2}^{2} + x_{0,2} \\
\end{matrix}
\right]\tag{7}
$$

&emsp;&emsp;可以看出$x_{1}$包含了原始特征$x_{0,1},x_{0,2}$从一阶到二阶的所有可能叉乘组合。

#### 5.2 泛化FMs
&emsp;&emsp;交叉网络共享参数共享的灵感和FM模型一样，在FM模型中，特征$x_{i}$和权重向量$v_{i}$相关联，交叉项$x_{i}x_{j}$的权重由$<v_{i},v_{j}>$计算，在DCN模型中，$x_{i}$和标量$[w_{k}^{i}]_{k=1}^{l}$相关联，并且$x_{i}x_{j}$的权重从集合$[w_{k}^{i}]_{k=1}^{l},[w_{k}^{j}]_{k=1}^{l}$中计算而来。

#### 5.3 有效映射
&emsp;&emsp;公式如下
$$x_{l+1} = x_{0}x_{l}^{T}w_{l} + b_{l} + x_{l} = f(x_{l},w_{l},b_{l}) + x_{l}$$

&emsp;&emsp;其中$x_{l}^{T}w_{l}$是一个标量，可以节省空间和时间。